In [ ]:
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Input Boxes and Dynamic Grouping Pie Charts")
        self.geometry("1920x1080")  # Fullscreen

        # Fullscreen window
        self.attributes('-fullscreen', True)
        self.bind("<Escape>", lambda event: self.attributes("-fullscreen", False))  # Press "Esc" to exit fullscreen

        # Input box for large values
        input_label = ttk.Label(self, text="Input:", font=("Arial", 12, "bold"))
        input_label.grid(row=0, column=0, padx=5, pady=5, sticky="W")

        self.numbers_entry = ttk.Entry(self, width=80)
        self.numbers_entry.grid(row=0, column=1, columnspan=9, padx=5, pady=5)

        # Labels for sets and grid for input boxes
        self.inputs = []
        for row in range(5):
            label = ttk.Label(self, text=f"Set {row+1}", font=("Arial", 10, "bold"))
            label.grid(row=row+1, column=0, padx=5, pady=5, sticky="W")

            input_row = []
            cols = 9 if row < 4 else 2  # 9 inputs for first 4 rows, 2 inputs for the last row
            for col in range(cols):
                entry = ttk.Entry(self, width=10)
                entry.grid(row=row+1, column=col+1, padx=5, pady=5)
                input_row.append(entry)
            self.inputs.append(input_row)

        # Grouping Section
        grouping_label = ttk.Label(self, text="Grouping:", font=("Arial", 12, "bold"))
        grouping_label.grid(row=6, column=0, padx=5, pady=10, sticky="W")

        self.group_frame = ttk.Frame(self)
        self.group_frame.grid(row=6, column=1, columnspan=9, padx=5, pady=5, sticky="W")

        # Initial Group 1
        self.groups = []
        self.add_group()

        # Button to add new groups
        self.add_group_button = ttk.Button(self, text="Add New Group", command=self.add_group)
        self.add_group_button.grid(row=7, column=0, columnspan=2, pady=10)

        # Button to generate the pie charts
        self.generate_button = ttk.Button(self, text="Generate Pie Charts", command=self.generate_pie_charts)
        self.generate_button.grid(row=7, column=2, columnspan=8, pady=10)

        # Placeholder for pie charts
        self.figure = plt.Figure(figsize=(8, 8), dpi=100)
        self.canvas = FigureCanvasTkAgg(self.figure, self)
        self.canvas.get_tk_widget().grid(row=0, column=10, rowspan=8, padx=10, pady=5)

    def add_group(self):
        """Add a new group with a "+" button to add sets."""
        group_num = len(self.groups) + 1

        group_label = ttk.Label(self.group_frame, text=f"Group {group_num}:", font=("Arial", 10, "bold"))
        group_label.grid(row=len(self.groups), column=0, padx=5, pady=5, sticky="W")

        group_entry_frame = ttk.Frame(self.group_frame)
        group_entry_frame.grid(row=len(self.groups), column=1, padx=5, pady=5, sticky="W")

        group_entries = []
        entry = ttk.Entry(group_entry_frame, width=5)
        entry.grid(row=0, column=0, padx=5)
        group_entries.append(entry)

        add_set_button = ttk.Button(group_entry_frame, text="+", command=lambda: self.add_set(group_entry_frame, group_entries))
        add_set_button.grid(row=0, column=1, padx=5)

        self.groups.append(group_entries)

    def add_set(self, frame, group_entries):
        """Add an additional input box to a group."""
        entry = ttk.Entry(frame, width=5)
        entry.grid(row=0, column=len(group_entries), padx=5)
        group_entries.append(entry)

    def generate_pie_charts(self):
        # Extract inputs
        input_data = [[int(entry.get()) if entry.get().isdigit() else 0 for entry in row] for row in self.inputs]
        user_numbers = list(map(int, self.numbers_entry.get().rstrip(',').split(',')))

        # Calculate match percentage for the first pie chart
        set_matches = [len([num for num in user_numbers if num in row]) for row in input_data]
        total_matches = sum(set_matches)
        set_percentages = [(matches / total_matches) * 100 if total_matches > 0 else 0 for matches in set_matches]

        # Calculate group percentages based on user-defined groups
        group_matches = []
        for group_entries in self.groups:
            group_sets = [int(entry.get()) - 1 for entry in group_entries if entry.get().isdigit() and 1 <= int(entry.get()) <= 5]
            group_match_sum = sum([set_matches[i] for i in group_sets])
            group_matches.append(group_match_sum)

        total_group_matches = sum(group_matches)
        group_percentages = [(matches / total_group_matches) * 100 if total_group_matches > 0 else 0 for matches in group_matches]

        # Clear the previous figure
        self.figure.clear()

        # Create first pie chart (Set match percentages)
        ax1 = self.figure.add_subplot(211)
        wedges1, texts1, autotexts1 = ax1.pie(set_percentages, labels=[f'Set {i+1}' for i in range(5)], autopct='%1.1f%%')
        ax1.set_title("Set Match Percentages")

        # Create second pie chart (Group match percentages)
        ax2 = self.figure.add_subplot(212)
        wedges2, texts2, autotexts2 = ax2.pie(group_percentages, labels=[f'Group {i+1}' for i in range(len(self.groups))], autopct='%1.1f%%')
        ax2.set_title("Group Percentages")

        # Hover effect (highlight wedge on hover)
        def on_hover(event, wedges, autotexts):
            for i, wedge in enumerate(wedges):
                if wedge.contains(event)[0]:
                    wedge.set_edgecolor('black')
                    wedge.set_linewidth(2)
                    autotexts[i].set_fontsize(14)
                else:
                    wedge.set_edgecolor('none')
                    wedge.set_linewidth(1)
                    autotexts[i].set_fontsize(10)
            self.canvas.draw()

        self.canvas.mpl_connect("motion_notify_event", lambda event: on_hover(event, wedges1, autotexts1))
        self.canvas.mpl_connect("motion_notify_event", lambda event: on_hover(event, wedges2, autotexts2))

        self.canvas.draw()

# Create the application window
app = App()
app.mainloop()


<ipython-input-18-380251f37ed6>:118: MatplotlibDeprecationWarning: normalize=None does not normalize if the sum is less than 1 but this behavior is deprecated since 3.3 until two minor releases later. After the deprecation period the default value will be normalize=True. To prevent normalization pass normalize=False 
  wedges2, texts2, autotexts2 = ax2.pie(group_percentages, labels=[f'Group {i+1}' for i in range(len(self.groups))], autopct='%1.1f%%')
